# Autotst experiments

This notebook contains the experiments with the autotst test.

The numbering of the experiments corresponds to the one in the file experiments.py.

The results are saved in the "results" directiory.

In [ ]:
import numpy as np
from seed import generate_seed
from sampling import f_theta_sampler
import autotst
from pathlib import Path
Path("results").mkdir(exist_ok=True, parents=True)
from mnist import load_mnist
P, Q_list = load_mnist()

In [ ]:
def sample_and_test_uniform(
    function_type, seed, kernel_type, approx_type, m, n, d, p, s, 
    perturbation_multiplier, alpha, l_minus, l_plus, B1, B2, B3, bandwidth_multipliers, number_bandwidths=10,
):  
    """
    Sample from uniform and perturbed uniform density and run two-sample test.
    inputs: function_type: "uniform", "increasing", "decreasing", "centred", "ost", 
                           "median", "split", "split (doubled sample sizes)" or "mmdagg_update"
            seed: integer random seed
            kernel_type: "gaussian" or "laplace": 
            approx_type: "permutation" (for MMD_a estimate Eq. (3)) 
                         or "wild bootstrap" (for MMD_b estimate Eq. (6))
            m: non-negative integer (sample size for uniform distribution)
            n: non-negative integer (sample size for perturbed uniform distribution)
            d: non-negative integer (dimension of samples)
            p: non-negative integer (number of permutations)
            s: positive number (smoothness parameter of Sobolev ball (Eq. (1))
            perturbation_multiplier: perturbation_multiplier: positive number (c_d in Eq. (17)) 
            alpha: real number in (0,1) (level of the test)
            l_minus: integer (for collection of bandwidths Eq. (16) in our paper)
            l_plus: integer (for collection of bandwidths Eq. (16) in our paper)
            B1: number of simulated test statistics to estimate the quantiles
            B2: number of simulated test statistics to estimate the probability in Eq. (13) in our paper
            B3: number of iterations for the bisection method
            bandwidth_multipliers: array such that mmd_split_test function (used for "split" 
                                   and "split (doubled sample sizes)") selects 'optimal' bandwidth from
                                   collection_bandwidths = [c*bandwidth_median for c in bandwidth_multipliers]
    output: result of test (1 for "REJECT H_0" and 0 for "FAIL TO REJECT H_0")
    """
    if function_type == "split (doubled sample sizes)":
        m = 2 * m
        n = 2 * n
    rs = np.random.RandomState(seed)
    if p == 0:
        X = rs.uniform(0, 1, (m, d)) 
        Y = rs.uniform(0, 1, (n, d))         
    else:
        X = f_theta_sampler(seed + 1, seed + 2, m, p, s, perturbation_multiplier, d)
        Y = rs.uniform(0, 1, (n, d))
    if function_type == "median":
        return mmd_median_test(
            seed, X, Y, alpha, kernel_type, approx_type, B1, bandwidth_multiplier=1
        )
    elif function_type in ["split", "split (doubled sample sizes)"]:
        return mmd_split_test(
            seed, X, Y, alpha, kernel_type, approx_type, B1, bandwidth_multipliers
        )
    elif function_type == "ost":
        return ost(seed, X, Y, alpha, kernel_type, l_minus, l_plus)
    elif function_type in ["uniform", "increasing", "decreasing", "centred"]:
        return mmdagg(
            seed, X, Y, alpha, kernel_type, approx_type, 
            function_type, l_minus, l_plus, B1, B2, B3
        )
    elif function_type == "mmdagg_update":
        if approx_type == "permutation":
            permutations_same_sample_size = True
        elif approx_type == "wild bootstrap" or approx_type == "wild_bootstrap": 
            permutations_same_sample_size = False
        else:
            raise ValueError('approx_type should be "permutation" or "wild bootstrap".')
        return mmdagg_update(
            X,
            Y,
            kernel=kernel_type,
            B1=B1,
            B2=B2,
            B3=B3,
            number_bandwidths=number_bandwidths,
            seed=seed,
            permutations_same_sample_size=permutations_same_sample_size,
        )
    elif function_type == "autotst":
        tst = autotst.AutoTST(X, Y)
        p_value = tst.p_value()
        output = int(p_value <= alpha)
        return output
    else:
        raise ValueError(
            'Undefined function_type: function_type should be "median", "split",' 
            '"split (doubled sample sizes)", "ost", "uniform", "increasing", '
            '"decreasing" or "centred".'
        )  

        
def sample_and_test_mnist(
    P, Q, function_type, seed, kernel_type, approx_type, m, n, 
    alpha, l_minus, l_plus, B1, B2, B3, bandwidth_multipliers, number_bandwidths=10,
):  
    """
    Sample from dataset P and dataset Q and run two-sample test.
    inputs: P: dataset of shape (number_points, dimension) from which to sample
            Q: dataset of shape (number_points, dimension) from which to sample
            function_type: "uniform", "increasing", "decreasing", "centred", "ost", 
                           "median", "split" or "split (doubled sample sizes)"
            seed: integer random seed
            kernel_type: "gaussian" or "laplace":
            approx_type: "permutation" (for MMD_a estimate Eq. (3)) 
                         or "wild bootstrap" (for MMD_b estimate Eq. (6))
            m: non-negative integer (sample size for uniform distribution)
            n: non-negative integer (sample size for perturbed uniform distribution)
            alpha: real number in (0,1) (level of the test)
            l_minus: integer (for collection of bandwidths Eq. (16) in our paper)
            l_plus: integer (for collection of bandwidths Eq. (16) in our paper)
            B1: number of simulated test statistics to estimate the quantiles
            B2: number of simulated test statistics to estimate the probability in Eq. (13) in our paper
            B3: number of iterations for the bisection method
            bandwidth_multipliers: array such that mmd_split_test function (used for "split" 
                                   and "split (doubled sample sizes)") selects 'optimal' bandwidth from
                                   collection_bandwidths = [c*bandwidth for c in bandwidth_multipliers]
    output: result of test (1 for "REJECT H_0" and 0 for "FAIL TO REJECT H_0")
    """
    rs = np.random.RandomState(seed)
    if function_type == "split (doubled sample sizes)":
        m = 2 * m
        n = 2 * n 
    idx_X = rs.randint(len(P), size=m)
    X = P[idx_X, :]
    idx_Y = rs.randint(len(Q), size=n)
    Y = Q[idx_Y, :]
    if function_type == "median":
        return mmd_median_test(
            seed, X, Y, alpha, kernel_type, approx_type, B1, bandwidth_multiplier=1
        )
    elif function_type in ["split", "split (doubled sample sizes)"]:
        return mmd_split_test(
            seed, X, Y, alpha, kernel_type, approx_type, B1, bandwidth_multipliers
        )
    elif function_type == "ost":
        return ost(seed, X, Y, alpha, kernel_type, l_minus, l_plus)
    elif function_type in ["uniform", "increasing", "decreasing", "centred"]:
        return mmdagg(
            seed, X, Y, alpha, kernel_type, approx_type, 
            function_type, l_minus, l_plus, B1, B2, B3
        )
    elif function_type == "mmdagg_update":
        if approx_type == "permutation":
            permutations_same_sample_size = True
        elif approx_type == "wild bootstrap" or approx_type == "wild_bootstrap": 
            permutations_same_sample_size = False
        else:
            raise ValueError('approx_type should be "permutation" or "wild bootstrap".')
        return mmdagg_update(
            X,
            Y,
            kernel=kernel_type,
            B1=B1,
            B2=B2,
            B3=B3,
            number_bandwidths=number_bandwidths,
            seed=seed,
            permutations_same_sample_size=permutations_same_sample_size,
        )
    elif function_type == "autotst":
        tst = autotst.AutoTST(X, Y)
        p_value = tst.p_value()
        output = int(p_value <= alpha)
        return output
    else:
        raise ValueError(
            'Undefined function_type: function_type should be "median", "split",' 
            '"split (doubled sample sizes)", "ost", "uniform", "increasing", '
            '"decreasing" or "centred".'
        )

# Experiment 1

Figure 3a, Figure 3b

In [ ]:
def experiment1(j, approx_type="wild bootstrap"):
    
    number_bandwidths = 10
    B1 = B2 = 2000
    B3 = 50
    kernel_types = ["autotst",]
    
    dataset = "uniform"
    bandwidth_multipliers = None
    sample_sizes = [500, 2000]
    N_epochs = 500
    alpha = 0.05
    delta = 1
    perturbation_multipliers = [2.7, 7.3]
    perturbations = [4, 3]
    R = [(0, 10), ]
    r_num = len(R)
    k_num = len(kernel_types)
    function_types = ["autotst", ]
    f_num = len(function_types)
    if approx_type == "wild bootstrap":
        app = "a"
    elif approx_type == "permutation":
        app = "b"
        
    ekr = [(e,k,r) for e in range(2) for k in range(k_num) for r in range(r_num)]
    e,k,r = ekr[j]

    d = e + 1
    p_num = perturbations[e]
    perturbation_multiplier = perturbation_multipliers[e]
    kernel_type = kernel_types[k]
    r_min, r_max = R[r]
    n = m = sample_sizes[e]
  
    print("sample size", sample_sizes[e])
    print(kernel_type)
    print(" ")

    jobs = [[[] for p in range(p_num)] for w in range(f_num)] 

    k = 1
    for w in range(f_num):
        function_type = function_types[w]
        for p in range(p_num):
            for i in range(N_epochs):
                seed = generate_seed(k, e, r, w, p, i)
                jobs[w][p].append(sample_and_test_uniform( 
                    function_type, 
                    seed, 
                    kernel_type, 
                    approx_type, 
                    m, n, d, p + 1, delta, perturbation_multiplier, 
                    alpha, r_min, r_max, B1, B2, B3, 
                    bandwidth_multipliers,
                    number_bandwidths=number_bandwidths,
                ))

    results = [[jobs[w][p] for p in range(p_num)] for w in range(f_num)] 
    power   = [[sum(results[w][p]) / N_epochs for p in range(p_num)] for w in range(f_num)]
    print(power)
 
    return power

In [1]:
i = 0
power = experiment1(i)
np.save("results/autotst_exp1_" + str(i) + ".npy", power[0])

sample size 500
autotst
 
[[1.0, 0.794, 0.344, 0.154]]


In [2]:
i = 1
power = experiment1(i)
np.save("results/autotst_exp1_" + str(i) + ".npy", power[0])

sample size 2000
autotst
 
[[1.0, 0.312, 0.074]]


# Experiment 2

Figure 3c

In [7]:
from mnist import load_mnist
P, Q_list = load_mnist()

In [15]:
# nope
@
def experiment2(j, approx_type="wild bootstrap"):
    
    number_bandwidths = 10
    B1 = B2 = 2000
    B3 = 50
    kernel_types = ["autotst",]
    
    dataset = "mnist"
    bandwidth_multipliers = None
    n = m = 500
    N_epochs = 500
    alpha = 0.05
    delta = 1
    R = [[(8,12), (10,14), (12,16)], [(10,14), (12,16), (14,18)]] 
    assert len(R[0]) == len(R[1])
    r_num = 1
    k_num = len(kernel_types)
    function_types = ["autotst", ]
    f_num = len(function_types)
    q_num = len(Q_list)
    if approx_type == "wild bootstrap":
        app = "a"
    elif approx_type == "permutation":
        app = "b"
        
    kr = [(k,r) for k in range(k_num) for r in range(r_num)]
    k,r = kr[j]

    kernel_type = kernel_types[k]
    r_min, r_max = 0, 1
    
    print("sample size", m)
    print(kernel_type)
    print(" ")

    jobs = [[[] for q in range(q_num)] for w in range(f_num)] 
    
    k = 1
    for q in range(q_num):
        for w in range(f_num):
            function_type = function_types[w]
            for i in range(N_epochs):
                seed = generate_seed(k, 2, r, w, q, i)
                jobs[w][q].append(sample_and_test_mnist(P, Q_list[q], function_type, seed, kernel_type, approx_type, m, n, 
                        alpha, r_min, r_max, B1, B2, B3, bandwidth_multipliers, number_bandwidths=number_bandwidths))
    
    results = [[jobs[w][q] for q in range(q_num)] for w in range(f_num)]
    power   = [[sum(results[w][q]) / N_epochs for q in range(q_num)] for w in range(f_num)]
    print(power)
    
    return power

In [3]:
i = 0
power = experiment2(i)
np.save("results/autotst_exp2_" + str(i) + ".npy", power[0])

sample size 500
autotst
 
[[1.0, 1.0, 0.992, 0.934, 0.312]]


# Experiment 5

In [19]:
def experiment5(j, approx_type="wild bootstrap"):
    
    number_bandwidths = 10
    B1 = B2 = 2000
    B3 = 50
    kernel_types = ["autotst",]
    
    dataset = "butucea"
    bandwidth_multipliers = np.linspace(0.1,1,10)
    sample_sizes = [500, 2000]
    N_epochs = 5000
    alpha = 0.05
    delta = 1
    perturbation_multipliers = [2.7, 7.3]
    perturbations = [4,3]
    R = [(-4,-0)]
    r_num = len(R)
    k_num = len(kernel_types)
    function_types = ["autotst",]
    f_num = len(function_types)
    if approx_type == "wild bootstrap":
        app = "a"
    elif approx_type == "permutation":
        app = "b"
        
    ekr = [(e,k,r) for e in range(2) for k in range(k_num) for r in range(r_num)]
    e,k,r = ekr[j]

    d = e + 1
    p_num = perturbations[e]
    perturbation_multiplier = perturbation_multipliers[e]
    kernel_type = kernel_types[k]
    r_min, r_max = R[r]
    n = m = sample_sizes[e]
    
    print("sample size", sample_sizes[e])
    print(kernel_type)
    print(" ")
    
    jobs = [[] for w in range(f_num)] 
    
    k = 1
    for w in range(f_num):
        function_type = function_types[w]
        for i in range(N_epochs):
            seed = generate_seed(k, e, r, w, 5, i) 
            jobs[w].append(sample_and_test_uniform( 
                function_type,
                seed,
                kernel_type,
                approx_type,
                m, n, d, 0, delta, perturbation_multiplier,
                alpha, r_min, r_max, B1, B2, B3,
                bandwidth_multipliers,
                number_bandwidths=number_bandwidths,
            ))
    
    results = [jobs[w] for w in range(f_num)] 
    power   = [sum(results[w]) / N_epochs for w in range(f_num)]
    print(power)

    return power

In [4]:
i = 0
power = experiment5(i)
np.save("results/autotst_exp5_" + str(i) + ".npy", power)

sample size 500
autotst
 
[0.0462]


In [5]:
i = 1
power = experiment5(i)
np.save("results/autotst_exp5_" + str(i) + ".npy", power)

sample size 2000
autotst
 
[0.051]


# Experiment 6

In [36]:
def experiment6(j, approx_type="wild bootstrap"):
    
    number_bandwidths = 10
    B1 = B2 = 2000
    B3 = 50
    kernel_types = ["autotst",]
    
    dataset = "mnist"
    bandwidth_multipliers = [2**i for i in range(10,21)]
    n = m = 500
    N_epochs = 5000
    alpha = 0.05
    delta = 1
    R = [[(10,14)], [(12,16)]] 
    assert len(R[0]) == len(R[1])
    r_num = len(R[0])
    k_num = len(kernel_types)
    function_types = ["autotst", ]
    f_num = len(function_types)
    q_num = len(Q_list)
    if approx_type == "wild bootstrap":
        app = "a"
    elif approx_type == "permutation":
        app = "b"
        
    kr = [(k,r) for k in range(k_num) for r in range(r_num)]
    k,r = kr[j]

    kernel_type = kernel_types[k]
    r_min, r_max = 1, 1
    
    print("sample size", m)
    print(kernel_type)
    print(" ")

    jobs = [[]  for w in range(f_num)] 
    
    k = 1
    for w in range(f_num):
        function_type = function_types[w]
        for i in range(N_epochs):
            seed = generate_seed(k, 2, r, w, 5, i)
            jobs[w].append(sample_and_test_mnist(P, P, function_type, seed, kernel_type, approx_type, m, n, 
                    alpha, r_min, r_max, B1, B2, B3, bandwidth_multipliers, number_bandwidths=number_bandwidths))
    
    results = [jobs[w] for w in range(f_num)] 
    power   = [sum(results[w]) / N_epochs for w in range(f_num)]
    print(power)
    
    return power

In [6]:
i = 0
power = experiment6(i)
np.save("results/autotst_exp6_" + str(i) + ".npy", power)

sample size 500
autotst
 
[0.0518]
